In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
# funciones de limpieza de texto

In [3]:
diccionario = {
    "😊": "felicidad",
    "😅": "risas",
    "🤣": "risas",
    "❤️": "amor",
    "😰": "preocupacion",
    "😎": "relajacion",
    "💪": "animo",
    "😉": "broma",
    "😍": "encantado",
    "😆": "risas",
    "😄": "risas",
    "😃": "risas",
    "😇": "feliz",
    "🙃": "sonrisa",
    "😌": "tranquilidad",
    "😜": "broma",
    "🤪": "alegria",
    "😫": "impotencia",
    "😩": "infelicidad",
    "😫": "frustración",
    "🙁": "triste",
    "😢": "muy triste",
    "🤬": "maldad",
    "😤": "enojada",
    "😡": "muy enojada",
    "😠": "molesto",
    "😨": "asustado",
    "😐": "neutral",
    "🤔": "dudoso",
    "😲": "sorpresa",
    "🤥": "mentira",
    "🙄": "indiferencia",
    "🤑": "dinero",
    "🙀": "sorprendido",
    "👍": "aprobación",
    "👎": "desaprobación",
    "👏": "admiración"
    
}

def reemplazarEmoji(sentence): 
    new_sntc = ""
    for word in sentence:
        if(word in diccionario.keys()):
            new_sntc += " " + diccionario[word] + " "
        else:
            new_sntc += word
    return new_sntc

print(reemplazarEmoji("Hoy estoy 😰"))

Hoy estoy  preocupacion 


In [4]:
def eliminarTildes(texto):
    a, b = 'áéíóúüÁÉÍÓÚÜ', 'aeiouuAEIOUU'
    trans = str.maketrans(a, b)                # maketrans  de python:crea un mapeo uno a uno de un personaje para su traducción
                                                #utilizable para el método translate ()
    return texto.translate(trans)              #transalate traduce funcion de python, devuelve una cadena donde cada carácter

eliminarTildes("áéí el niño está en el avión")

'aei el niño esta en el avion'

In [5]:
def descomponerHastag(texto):
    
    if(texto[0]!="#"):
        return texto
    
    else:
        texto = texto[1:]
        
        if(texto == texto.upper()):           #COVID -> covid
            return texto.lower()
                                             
        else:                                #HolaPeru -> " Hola Peru"

            new_word = ""
            for x in texto:
                if( 65 <= ord(x) <= 90 ): # si es mayuscula     #ord, es el valor en sistema unicode.
                    new_word += " " + x
                else:
                    new_word += x
            return new_word.strip()       #strip es para elimina eliminar los espacios al inicio o al final
        
print(descomponerHastag("#YoMeQuedoEnCasa"), descomponerHastag("#100DíasDeHambreYMuerte"))

Yo Me Quedo En Casa 100 Días De Hambre Y Muerte


In [6]:
# leer la lista de stopwords
file = open("recursos/otros/stopwords.txt","r",encoding="utf-8")
stop_words = list(line[:-1] for line in file)
file.close()

In [7]:
# cambiar las tilder

def limpiarTexto(texto):
    
    texto = re.sub('https://[./\w]+', '', texto)               # eliminar urls
    texto = reemplazarEmoji(texto)                             # reemplazar emojis
    texto = eliminarTildes(texto)                              # reemplazar las tildes
    lista = re.findall("[#@a-zA-Zñáéíóú]+", texto)             # identificar palabras en español, nos quedamos solo con palabras español
    lista = [ x for x in lista if x[0] != "@" ]                # eliminar los nombres de usuarios de twitter @Mario
    lista = [ descomponerHastag(x) for x in lista ]            # descomponer los hashtags
    texto = " ".join(list(lista))                              # concatenar la lista
    
    lista = re.findall("[#@a-zA-Zñáéíóú]+", texto)             # identificar palabras en español
    lista = list(filter(lambda x: x not in stop_words, lista)) # eliminar stopwords  # porque eso vocabulario noe es importante para nustro analisis
    lista = filter(lambda x: len(x) > 2, lista)                # eliminar rt, q, xq, ...str>2
    texto = " ".join(list(lista))                              # concatenar la lista 
    texto = texto.strip()                                      # eliminar espacios innecesarios
    texto = texto.lower()                                      # todo a minusculas
    
    return texto

In [10]:
# leer la base de datos de tweets
data = pd.read_csv("recursos/prediccion/bd_tweets.csv")

In [11]:
data.sample(5)

,fecha,texto
6340,2020-04-24 22:12:18,Presidente #Vizcarra por que el gas doméstico ...
3333,2020-03-19 22:13:01,ÚLTIMO MINUTO: @Minsa_Peru\n confirma primera ...
8528,2020-04-11 13:54:11,"Soy sólo yo, o todo estamos durmiendo muy mal ..."
13656,2020-05-08 18:59:47,#Vizcarra Lunes 18: niños con acompañantes pod...
9690,2020-04-19 15:34:39,Este es el tipo de análisis que requerimos de ...


In [12]:
# inspeccionar la limpieza
res = data["texto"].sample(5).apply(lambda x: limpiarTexto(x))

for x in res: print(x + "\n")

culpa vizcarra

primero desobedecen ordenes simples salir fiesta asustan futuras medidas seguridad coronavirusperu

fia ayuda paliar situacion muerte contagio coronavirusperu porque congreso ayuda situacion emergencia queden quietos despues pandemia interesa muertos contagiados

peruanos cojudos peru ahorcado economicamente cuarentena infinita vizcarra ustedes mas pendientes ocurre estados unidos

mujeresam esta gente entiende gobierno advierte los ambulantes aahh ate caso mano dura vizcarra quedate casa contagio cuarentena



In [13]:
# almacenar los tweets limpios
data["texto_limpio"] = data["texto"].apply(lambda x: limpiarTexto(x))

In [14]:
data.sample(5)

,fecha,texto,texto_limpio
16090,2020-06-29 02:33:23,[Video] #PanamericanaNoticias #Panorama Contra...,video panamericana noticias panorama contratac...
8085,2020-04-20 18:29:24,Qué bien!!! Presidente #vizcarra anuncia que l...,que presidente vizcarra anuncia colegios parti...
5483,2020-03-07 12:14:44,"A todos los borrachos, ya saben a usar sorbete...",borrachos sorbete coronavirusperu
9143,2020-04-07 20:13:29,"Coronavirus | Qué es la ""doble curva"" y por qu...",coronavirus que doble curva falso dilema elegi...
18199,2020-06-25 12:06:34,#Día398 para el cambio de presidente. \n\nIncr...,dia cambio presidente increible donaciones usa...


In [15]:
# Guardar los tweets limpios en un archivo csv
np.savetxt("recursos/prediccion/bd_tweets_limpios.csv",
           data["texto_limpio"].values,
           header = "texto_limpio",
           fmt="%s", 
           encoding="utf-8")

In [16]:
# leer la base de datos de comentarios del banco
data_bcp = pd.read_csv("recursos/entrenamiento/comentarios.csv")

In [17]:
data_bcp.sample(5)

,score,comment
7314,3,un poco mas de orden en la agencia
4478,4,cuando fui no habia mucha gente realice mi ope...
1336,5,por la rapidez en la atencion
7788,3,mejor asesoramiento en cuanto a sus intrumento...
336,5,atencion rapida y adecuada


In [18]:
# guardar los comentarios limpios
data_bcp["texto_limpio"] = data_bcp["comment"].apply(lambda x: limpiarTexto(x))

In [19]:
data_bcp.sample(5)

,score,comment,texto_limpio
12031,3,trato mas personalizado no apurados,trato mas personalizado apurados
7249,2,solo habian personas atendiendo y eramos bast...,habian personas atendiendo bastantes personas ...
9928,3,aperturar mas ventanillas para la atencion a l...,aperturar mas ventanillas atencion visitantes
10508,3,mejorar el tiempo para la atencion,mejorar atencion
2730,5,me atendieron eficazmente,atendieron eficazmente


In [20]:
# Guardar los comentarios limpios en un archivo csv
np.savetxt("recursos/entrenamiento/comentarios_limpios.csv",
           data_bcp["texto_limpio"].values,
           header = "texto_limpio",
           fmt="%s", 
           encoding="utf-8")